<h2 style='color:purple' align='center'>Naive Bayes Tutorial Part 1: Predicting survival from titanic crash</h2>

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("/kaggle/input/titanic-data/titanic.csv")
df.head()

,PassengerId,Name,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,1,"Braund, Mr. Owen Harris",3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,"Heikkinen, Miss. Laina",3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,female,35.0,1,0,113803,53.1000,C123,S,1
4,5,"Allen, Mr. William Henry",3,male,35.0,0,0,373450,8.0500,NaN,S,0


In [3]:
df.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Embarked'],axis='columns',inplace=True)
df.head()

,Pclass,Sex,Age,Fare,Survived
0,3,male,22.0,7.2500,0
1,1,female,38.0,71.2833,1
2,3,female,26.0,7.9250,1
3,1,female,35.0,53.1000,1
4,3,male,35.0,8.0500,0


In [4]:
inputs = df.drop('Survived',axis='columns')
target = df.Survived

In [5]:
#inputs.Sex = inputs.Sex.map({'male': 1, 'female': 2})

In [6]:
dummies = pd.get_dummies(inputs.Sex)
dummies.head(3)

,female,male
0,0,1
1,1,0
2,1,0


In [7]:
inputs = pd.concat([inputs,dummies],axis='columns')
inputs.head(3)

,Pclass,Sex,Age,Fare,female,male
0,3,male,22.0,7.2500,0,1
1,1,female,38.0,71.2833,1,0
2,3,female,26.0,7.9250,1,0


**I am dropping male column as well because of dummy variable trap theory. One column is enough to repressent male vs female**

In [8]:
inputs.drop(['Sex','male'],axis='columns',inplace=True)
inputs.head(3)

,Pclass,Age,Fare,female
0,3,22.0,7.2500,0
1,1,38.0,71.2833,1
2,3,26.0,7.9250,1


In [9]:
inputs.columns[inputs.isna().any()]

Index(['Age'], dtype='object')

In [10]:
inputs.Age[:10]

0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
5     NaN
6    54.0
7     2.0
8    27.0
9    14.0
Name: Age, dtype: float64

In [11]:
inputs.Age = inputs.Age.fillna(inputs.Age.mean())
inputs.head()

,Pclass,Age,Fare,female
0,3,22.0,7.2500,0
1,1,38.0,71.2833,1
2,3,26.0,7.9250,1
3,1,35.0,53.1000,1
4,3,35.0,8.0500,0


In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs,target,test_size=0.3)

In [13]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

In [14]:
model.fit(X_train,y_train)

GaussianNB()

In [15]:
model.score(X_test,y_test)

0.753731343283582

In [16]:
X_test[0:10]

,Pclass,Age,Fare,female
47,3,29.699118,7.7500,1
871,1,47.000000,52.5542,1
599,1,49.000000,56.9292,0
629,3,29.699118,7.7333,0
456,1,65.000000,26.5500,0
426,2,28.000000,26.0000,1
305,1,0.920000,151.5500,0
817,2,31.000000,37.0042,0
24,3,8.000000,21.0750,1
464,3,29.699118,8.0500,0


In [17]:
y_test[0:10]

47     1
871    1
599    1
629    0
456    0
426    1
305    1
817    0
24     0
464    0
Name: Survived, dtype: int64

In [18]:
model.predict(X_test[0:10])

array([1, 1, 0, 0, 0, 1, 1, 0, 1, 0])

In [19]:
model.predict_proba(X_test[:10])

array([[0.33502693, 0.66497307],
       [0.03604674, 0.96395326],
       [0.65248719, 0.34751281],
       [0.9666161 , 0.0333839 ],
       [0.72649417, 0.27350583],
       [0.17612481, 0.82387519],
       [0.00272923, 0.99727077],
       [0.91716154, 0.08283846],
       [0.22629667, 0.77370333],
       [0.96663984, 0.03336016]])

**Calculate the score using cross validation**

In [20]:
from sklearn.model_selection import cross_val_score
cross_val_score(GaussianNB(),X_train, y_train, cv=5)

array([0.824     , 0.752     , 0.752     , 0.79032258, 0.81451613])